In [10]:
from pydantic import BaseModel, Field, field_validator, ValidationError

In [2]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

In [3]:
Model(number="4")

Model(number=4)

In [4]:
Model(number="12")

ValidationError: 1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value='12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/less_than

In [5]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)
    
    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        return value

In [6]:
Model(number=3)

Running custom validator
value=3, type(value)=<class 'int'>


Model(number=3)

In [7]:
Model(number="3")

Running custom validator
value=3, type(value)=<class 'int'>


Model(number=3)

In [8]:
Model(number=12)

ValidationError: 1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/less_than

In [11]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)
    
    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        if value % 2 == 0:
            return value
        raise ValueError("Value must be even")

In [13]:
Model(number=4)

Running custom validator
value=4, type(value)=<class 'int'>


Model(number=4)

In [14]:
Model(number=3)

Running custom validator
value=3, type(value)=<class 'int'>


ValidationError: 1 validation error for Model
number
  Value error, Value must be even [type=value_error, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [15]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)
    
    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        if value % 2 == 0:
            return value
        raise TypeError("Value must be even")

In [16]:
Model(number=3)

Running custom validator
value=3, type(value)=<class 'int'>


TypeError: Value must be even

In [17]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)
    
    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        if value % 2 == 1:
            return value + 1
        return value

In [19]:
Model(number=3), Model(number=2)

(Model(number=4), Model(number=2))

In [31]:
from datetime import datetime
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [32]:
datetime.now()

datetime.datetime(2024, 8, 28, 12, 45, 25, 992701)

In [33]:
make_utc(datetime.now())

datetime.datetime(2024, 8, 28, 12, 45, 26, 315301, tzinfo=<UTC>)

In [49]:
class Model(BaseModel):
    dt: datetime
    
    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

In [50]:
Model(dt="2020-01-01T03:00:00")

Model(dt=datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>))

In [51]:
eastern = pytz.timezone("US/Eastern")

In [52]:
dt = eastern.localize(datetime(2020, 1, 1, 3, 0, 0))

In [53]:
dt

datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [54]:
Model(dt=dt)

Model(dt=datetime.datetime(2020, 1, 1, 8, 0, tzinfo=<UTC>))

In [57]:
class Model(BaseModel):
    number: int
    
    @field_validator("number")
    @classmethod
    def add_1(cls, value: int):
        print(f"Running add_1({value}) -> {value + 1}")
        return value + 1
    
    @field_validator("number")
    @classmethod
    def add_2(cls, value: int):
        print(f"Running add_2({value}) -> {value + 2}")
        return value + 2

In [58]:
Model(number=0)

Running add_1(0) -> 1
Running add_2(1) -> 3


Model(number=3)

In [59]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float
    
    @field_validator("unit_cost", "unit_price")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

In [60]:
Model(unit_cost=2.12345, unit_price=5.9876)

Model(unit_cost=2.12, unit_price=5.99)

In [61]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float
    
    @field_validator("*")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

In [62]:
Model(unit_cost=2.12345, unit_price=5.9876)

Model(unit_cost=2.12, unit_price=5.99)